In [6]:
import torch
from torch import nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


In [8]:
#超参
EPOCH = 1               # train the training data n times, to save time, we just train 1 epoch
BATCH_SIZE = 64         #每次每批训练的数量
TIME_STEP = 28          # rnn time step / image height （考虑时间点数据，每步）
INPUT_SIZE = 28         # rnn input size / image width
LR = 0.01               # learning rate
DOWNLOAD_MNIST = False   # set to True if haven't download the data

train_data = dsets.MNIST(
    root='./mnist/',
    train=True,                         # this is training data
    transform=transforms.ToTensor(),    # Converts a PIL.Image or numpy.ndarray to
                                        # torch.FloatTensor of shape (C x H x W) and normalize in the range [0.0, 1.0]
    download=DOWNLOAD_MNIST,            # download it if you don't have it
)

# Data Loader for easy mini-batch return in training
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

# convert test data into Variable, pick 2000 samples to speed up testing
test_data = dsets.MNIST(root='./mnist/', train=False, transform=transforms.ToTensor())
test_x = test_data.test_data.type(torch.FloatTensor)[:2000]/255.   # shape (2000, 28, 28) value in range(0,1)
test_y = test_data.test_labels.numpy()[:2000]    # covert to numpy array

class RNN(nn.Module):
    def __init__(self) -> None:
        super(RNN,self).__init__()

        self.rnn = nn.LSTM(
            input_size = INPUT_SIZE,  #每个时间点上input INPUT_SIZE个像素点
            hidden_size = 64, #
            num_layers = 1,   #一个隐藏层中的神经元数
            batch_first = True,  #输入数据的第一维度为batch_size

        )
        self.out = nn.Linear(64,10)


    def forward(self,x):
        #RNN的输出，隐藏层的状态（主线程和分线层）  None represents zero initial hidden state
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, (h_n, h_c) = self.rnn(x, None)   

        # choose r_out at the last time step
        out = self.out(r_out[:, -1, :])
        return out

rnn = RNN()
print(rnn)

optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   # optimize all cnn parameters
#target标签是数字，而不是one-hot编码
loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted

# training and testing
for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(train_loader):        # gives batch data
        b_x = b_x.view(-1, 28, 28)              # reshape x to (batch, time_step, input_size)

        output = rnn(b_x)                               # rnn output
        loss = loss_func(output, b_y)                   # cross entropy loss
        optimizer.zero_grad()                           # clear gradients for this training step
        loss.backward()                                 # backpropagation, compute gradients
        optimizer.step()                                # apply gradients

        if step % 50 == 0:
            test_output = rnn(test_x)                   # (samples, time_step, input_size)
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == test_y).astype(int).sum()) / float(test_y.size)
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)

# print 10 predictions from test data
test_output = rnn(test_x[:10].view(-1, 28, 28))
pred_y = torch.max(test_output, 1)[1].data.numpy()
print(pred_y, 'prediction number')
print(test_y[:10], 'real number')



RNN(
  (rnn): LSTM(28, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
)
Epoch:  0 | train loss: 2.3078 | test accuracy: 0.18
Epoch:  0 | train loss: 0.8463 | test accuracy: 0.62
Epoch:  0 | train loss: 0.7100 | test accuracy: 0.71
Epoch:  0 | train loss: 0.6754 | test accuracy: 0.80
Epoch:  0 | train loss: 0.3142 | test accuracy: 0.86
Epoch:  0 | train loss: 0.5702 | test accuracy: 0.88
Epoch:  0 | train loss: 0.3486 | test accuracy: 0.90
Epoch:  0 | train loss: 0.2996 | test accuracy: 0.91
Epoch:  0 | train loss: 0.2149 | test accuracy: 0.93
Epoch:  0 | train loss: 0.0482 | test accuracy: 0.93
Epoch:  0 | train loss: 0.0404 | test accuracy: 0.94
Epoch:  0 | train loss: 0.1969 | test accuracy: 0.94
Epoch:  0 | train loss: 0.1791 | test accuracy: 0.93
Epoch:  0 | train loss: 0.1750 | test accuracy: 0.95
Epoch:  0 | train loss: 0.1932 | test accuracy: 0.95
Epoch:  0 | train loss: 0.1329 | test accuracy: 0.96
Epoch:  0 | train loss: 0.0545 | test accurac